## Load necessary modules

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import pandas as pd

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


## Load RetinaNet model

In [2]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('/data/a2301133/RetinaNet/snapshots/', 'resnet50_csv_50.h5')

# load retinanet model
# model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
model = models.load_model(model_path, backbone_name='resnet50', convert=True)

# print(model.summary())

In [3]:
# load label to names mapping for visualization purposes
# labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}
labels_to_names = {}
with open("SF_mapping.txt", "rb") as f:
    for line in f:
        data = line.strip().split(',')
        labels_to_names[int(data[1])] = data[0]
f.close()

## Run detection on example

In [6]:
image_folder = '/project/ibm/a2301133/3videoFRCNN/198/2018-02-01_16-27-59-198/'
images = os.listdir(image_folder)
images = [x for x in images if '.jpg' in x]
threshold = 0.5
n = len(images)
i = 0
with open("198_result_new.txt", "wb") as f:
    for img_path in images:
        image = read_image_bgr(image_folder + img_path)
        image = preprocess_image(image)
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
        n = len(scores)
        m = len(scores[0])
        for i in xrange(n):
            for j in xrange(m):
                if scores[i][j] >= threshold:
                    xmin, ymin, xmax, ymax = boxes[i][j][0], boxes[i][j][1], boxes[i][j][2], boxes[i][j][3]
                    bbox = str(xmin) + ',' + str(ymin) + ',' + str(xmax) + ',' + str(ymax)
                    f.write(str(img_path) +',' + str(bbox) + ',' + str(labels[i][j]) + ',' + str(scores[i][j]) + '\n')
        i += 1
f.close()

In [11]:
import json
frames_list = []
txt_file = open("198_result_new.txt" , 'rb')
output_path = "198_result_new.json"
for line in txt_file:
    line = line.strip().split(',')
    frame_num = line[0]
    f_dict = {'frame_number': str(frame_num), 'RoIs': ''}
    xmin, ymin, xmax, ymax = int(float(line[1])), int(float(line[2])), int(float(line[3])), int(float(line[4]))
    delta_x, delta_y = xmax - xmin, ymax - ymin
    xmin, ymin, delta_x, delta_y = str(xmin), str(ymin), str(delta_x), str(delta_y)
    rois_list = [xmin, ymin, delta_x, delta_y, labels_to_names.get(int(line[5]))]
    if len(rois_list) > 0:
        f_dict['RoIs'] = ','.join(rois_list) + ';'
    frames_list.append(f_dict)

json_dict = {'output': {'frames': frames_list}}
with open(output_path, 'wb') as out_file:
    json.dump(json_dict, out_file, indent = 4)


In [25]:
len(data)

2538

In [ ]:
data = os.listdir(image_folder)
data = [x for x in data if '.jpg' in x]
data.sort()
# data = pd.read_csv("198_result_new.txt", header = None)
# data = ['0009660.jpg']

for i in range(0, len(data)):
    image = read_image_bgr(image_folder + data[i])
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    
    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis = 0))
    print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)


    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(draw)
    plt.savefig('/nethome/cwu392/Keras/keras-retinanet/198/' + data[i])
#     plt.show()
    plt.close()

('processing time: ', 0.1354529857635498)
('processing time: ', 0.13251280784606934)
('processing time: ', 0.13322210311889648)
('processing time: ', 0.13344812393188477)
('processing time: ', 0.139024019241333)
('processing time: ', 0.14059901237487793)
('processing time: ', 0.13670611381530762)
('processing time: ', 0.1365039348602295)
('processing time: ', 0.13168597221374512)
('processing time: ', 0.13782095909118652)
('processing time: ', 0.13665008544921875)
('processing time: ', 0.13135886192321777)
('processing time: ', 0.13196086883544922)
('processing time: ', 0.13898205757141113)
('processing time: ', 0.13809895515441895)
('processing time: ', 0.13686585426330566)
('processing time: ', 0.13459300994873047)
('processing time: ', 0.1313459873199463)
('processing time: ', 0.13504981994628906)
('processing time: ', 0.13821697235107422)
('processing time: ', 0.13823413848876953)
('processing time: ', 0.13289189338684082)
('processing time: ', 0.13452410697937012)
('processing tim

('processing time: ', 0.13338804244995117)
('processing time: ', 0.13222718238830566)
('processing time: ', 0.13271093368530273)
('processing time: ', 0.13380098342895508)
('processing time: ', 0.13671398162841797)
('processing time: ', 0.1334819793701172)
('processing time: ', 0.1306898593902588)
('processing time: ', 0.1328871250152588)
('processing time: ', 0.13519692420959473)
('processing time: ', 0.12941908836364746)
('processing time: ', 0.13825201988220215)
('processing time: ', 0.13381600379943848)
('processing time: ', 0.12925100326538086)
('processing time: ', 0.13496685028076172)
('processing time: ', 0.14521503448486328)
('processing time: ', 0.13363909721374512)
('processing time: ', 0.13378000259399414)
('processing time: ', 0.1319899559020996)
('processing time: ', 0.1334218978881836)
('processing time: ', 0.1325840950012207)
('processing time: ', 0.13201594352722168)
('processing time: ', 0.13048505783081055)
('processing time: ', 0.13164114952087402)
('processing time

('processing time: ', 0.13327407836914062)
('processing time: ', 0.14076590538024902)
('processing time: ', 0.1339280605316162)
('processing time: ', 0.13180184364318848)
('processing time: ', 0.13451814651489258)
('processing time: ', 0.13228893280029297)
('processing time: ', 0.12969708442687988)
('processing time: ', 0.13173484802246094)
('processing time: ', 0.1352381706237793)
('processing time: ', 0.13632893562316895)
('processing time: ', 0.1321868896484375)
('processing time: ', 0.1364760398864746)
('processing time: ', 0.13344097137451172)
('processing time: ', 0.13282084465026855)
('processing time: ', 0.13169598579406738)
('processing time: ', 0.13659000396728516)
('processing time: ', 0.1356339454650879)
('processing time: ', 0.13451480865478516)
('processing time: ', 0.13027501106262207)
('processing time: ', 0.1311788558959961)
('processing time: ', 0.13199210166931152)
('processing time: ', 0.13175106048583984)
('processing time: ', 0.13180994987487793)
('processing time

('processing time: ', 0.13357281684875488)
('processing time: ', 0.13327503204345703)
('processing time: ', 0.13233208656311035)
('processing time: ', 0.13446807861328125)
('processing time: ', 0.12960505485534668)
('processing time: ', 0.13388299942016602)
('processing time: ', 0.13336181640625)
('processing time: ', 0.13417792320251465)
('processing time: ', 0.13264703750610352)
('processing time: ', 0.1381549835205078)
('processing time: ', 0.12943005561828613)
('processing time: ', 0.13205289840698242)
('processing time: ', 0.13827800750732422)
('processing time: ', 0.13393497467041016)
('processing time: ', 0.13382911682128906)
('processing time: ', 0.13405203819274902)
('processing time: ', 0.1307680606842041)
('processing time: ', 0.1332540512084961)
('processing time: ', 0.1385488510131836)
('processing time: ', 0.13392305374145508)
('processing time: ', 0.13478398323059082)
('processing time: ', 0.13663196563720703)
('processing time: ', 0.1344599723815918)
('processing time: 

('processing time: ', 0.1347370147705078)
('processing time: ', 0.13503098487854004)
('processing time: ', 0.13332605361938477)
('processing time: ', 0.13266801834106445)
('processing time: ', 0.13385486602783203)
('processing time: ', 0.12916803359985352)
('processing time: ', 0.13109803199768066)
('processing time: ', 0.13254785537719727)
('processing time: ', 0.14137601852416992)
('processing time: ', 0.1470170021057129)
('processing time: ', 0.1319429874420166)
('processing time: ', 0.1353600025177002)
('processing time: ', 0.13045001029968262)
('processing time: ', 0.13332200050354004)
('processing time: ', 0.13344979286193848)
('processing time: ', 0.13406896591186523)
('processing time: ', 0.13846206665039062)
('processing time: ', 0.1392660140991211)
('processing time: ', 0.13031411170959473)
('processing time: ', 0.13384604454040527)
('processing time: ', 0.13341999053955078)
('processing time: ', 0.13547897338867188)
('processing time: ', 0.13625311851501465)
('processing tim

...

('processing time: ', 0.13181781768798828)
('processing time: ', 0.13499689102172852)
('processing time: ', 0.1369788646697998)
('processing time: ', 0.15056395530700684)
('processing time: ', 0.1377580165863037)
('processing time: ', 0.13282012939453125)
('processing time: ', 0.13285303115844727)
('processing time: ', 0.13526010513305664)
('processing time: ', 0.13460183143615723)
('processing time: ', 0.13270807266235352)
('processing time: ', 0.14008116722106934)
('processing time: ', 0.13679194450378418)
('processing time: ', 0.12544012069702148)
('processing time: ', 0.12711596488952637)
('processing time: ', 0.12572312355041504)
('processing time: ', 0.12717580795288086)
('processing time: ', 0.12398099899291992)
('processing time: ', 0.12844514846801758)
('processing time: ', 0.12902092933654785)
('processing time: ', 0.12677884101867676)
('processing time: ', 0.1270461082458496)
('processing time: ', 0.12937092781066895)
('processing time: ', 0.12952899932861328)
('processing t

('processing time: ', 0.1400771141052246)
('processing time: ', 0.13683605194091797)
('processing time: ', 0.1394197940826416)
('processing time: ', 0.1329669952392578)
('processing time: ', 0.13670897483825684)
('processing time: ', 0.13357210159301758)
('processing time: ', 0.13523221015930176)
('processing time: ', 0.13711094856262207)
('processing time: ', 0.13568806648254395)
('processing time: ', 0.13683700561523438)
('processing time: ', 0.1339101791381836)
('processing time: ', 0.1348259449005127)
('processing time: ', 0.13451695442199707)
('processing time: ', 0.1320199966430664)
('processing time: ', 0.13064098358154297)
('processing time: ', 0.13309907913208008)
('processing time: ', 0.13814187049865723)
('processing time: ', 0.13753104209899902)
('processing time: ', 0.13619613647460938)
('processing time: ', 0.13454794883728027)
('processing time: ', 0.1339249610900879)
('processing time: ', 0.1354508399963379)
('processing time: ', 0.13637185096740723)
('processing time: 

('processing time: ', 0.13374900817871094)
('processing time: ', 0.13310503959655762)
('processing time: ', 0.1292259693145752)
('processing time: ', 0.13880014419555664)
('processing time: ', 0.13597798347473145)
('processing time: ', 0.13971209526062012)
('processing time: ', 0.13572096824645996)
('processing time: ', 0.1340188980102539)
('processing time: ', 0.13343191146850586)
('processing time: ', 0.1299269199371338)
('processing time: ', 0.13129401206970215)
('processing time: ', 0.13519597053527832)
('processing time: ', 0.13310599327087402)
('processing time: ', 0.14464187622070312)
('processing time: ', 0.14696598052978516)
('processing time: ', 0.13307905197143555)
('processing time: ', 0.13785505294799805)
('processing time: ', 0.13348984718322754)
('processing time: ', 0.13328886032104492)
('processing time: ', 0.1324319839477539)
('processing time: ', 0.13549590110778809)
('processing time: ', 0.13686418533325195)
('processing time: ', 0.13489603996276855)
('processing ti

('processing time: ', 0.13381314277648926)
('processing time: ', 0.133681058883667)
('processing time: ', 0.13480305671691895)
('processing time: ', 0.13469505310058594)
('processing time: ', 0.1324751377105713)
('processing time: ', 0.13279986381530762)
('processing time: ', 0.1383988857269287)
('processing time: ', 0.1364438533782959)
('processing time: ', 0.13851714134216309)
('processing time: ', 0.13613486289978027)
('processing time: ', 0.13481402397155762)
('processing time: ', 0.13556885719299316)
('processing time: ', 0.13292694091796875)
('processing time: ', 0.12435722351074219)
('processing time: ', 0.13462090492248535)
('processing time: ', 0.1342179775238037)
('processing time: ', 0.12874794006347656)
('processing time: ', 0.12708377838134766)
('processing time: ', 0.12914609909057617)
('processing time: ', 0.13483095169067383)
('processing time: ', 0.1315009593963623)
('processing time: ', 0.13225221633911133)
('processing time: ', 0.1335439682006836)
('processing time: 

('processing time: ', 0.13524794578552246)
('processing time: ', 0.13505005836486816)
('processing time: ', 0.13524198532104492)
('processing time: ', 0.13289690017700195)
('processing time: ', 0.13311314582824707)
('processing time: ', 0.13485193252563477)
('processing time: ', 0.13228988647460938)
('processing time: ', 0.13411402702331543)
('processing time: ', 0.13176417350769043)
('processing time: ', 0.13047099113464355)
('processing time: ', 0.13274884223937988)
('processing time: ', 0.13244891166687012)
('processing time: ', 0.1490039825439453)
('processing time: ', 0.13965415954589844)
('processing time: ', 0.13515090942382812)
('processing time: ', 0.13221311569213867)
('processing time: ', 0.1323108673095703)
('processing time: ', 0.13573098182678223)
('processing time: ', 0.1362931728363037)
('processing time: ', 0.1305069923400879)
('processing time: ', 0.13093185424804688)
('processing time: ', 0.13650798797607422)
('processing time: ', 0.13416290283203125)
('processing ti

In [ ]:
import cv2
import os

image_folder = '/nethome/cwu392/Keras/keras-retinanet/198/'
video_name = '198.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
images.sort()
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape
frame = cv2.resize(frame, (width,height))
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
video = cv2.VideoWriter(video_name, fourcc, 5, (width,height))

for image in images:
    current_image = cv2.imread(os.path.join(image_folder, image))
    video.write(current_image)

cv2.destroyAllWindows()
video.release()